In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0


In [3]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])


In [4]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.98:
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [6]:
model.fit(train_images, train_labels, epochs=100, batch_size=32, callbacks=callbacks)


Epoch 1/100
1875/1875 [==============================] - 11s 5ms/step - loss: 0.4930 - accuracy: 0.8274
Epoch 2/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3713 - accuracy: 0.8659
Epoch 3/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3347 - accuracy: 0.8774
Epoch 4/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3098 - accuracy: 0.8861
Epoch 5/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2928 - accuracy: 0.8920
Epoch 6/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2783 - accuracy: 0.8976
Epoch 7/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2666 - accuracy: 0.9003
Epoch 8/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2560 - accuracy: 0.9050
Epoch 9/100
1875/1875 [==============================] - 12s 6ms/step - loss: 0.2485 - accuracy: 0.9074
Epoch 10/100
1875/1875 [==============================] - 12s 6ms/step

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


313/313 [==============================] - 2s 5ms/step - loss: 0.8014 - accuracy: 0.8851
Test accuracy: 0.8851000070571899


In [8]:
# Simpan model
model.save('image_detection_fmnistonly.h5')

In [13]:
import cv2

class_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Path ke gambar yang akan diprediksi
image_path = 'gambarTest/4.jpg'

# Membaca gambar
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
image = cv2.resize(image, (28, 28))
image = np.expand_dims(image, axis=-1)  
image = image.astype('float32') / 255

# Melakukan prediksi
predictions = model.predict(np.array([image]))
predicted_class = np.argmax(predictions)

# Mengonversi kelas prediksi menjadi label
predicted_label = class_labels[predicted_class]

# Menampilkan hasil prediksi
print('Predicted Label:', predicted_label)


1/1 [==============================] - 0s 136ms/step
Predicted Label: Bag


In [15]:
import os
from PIL import Image

# Load the trained model
model = keras.models.load_model('image_detection_fmnistonly.h5')

# Load the class labels
class_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Folder path
folder_path = 'gambarTest'  
# Process each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg'):
        # Load and preprocess the image
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path).convert('L')  # Convert image to grayscale
        image = image.resize((28, 28))  # Resize the image to match the input shape of the model
        image = np.array(image)  # Convert the image to a NumPy array
        image = image / 255.0  # Normalize the pixel values

        # Make a prediction
        prediction = model.predict(np.expand_dims(image, axis=0))
        predicted_label = np.argmax(prediction[0])

        # Print the predicted label
        print(f'Image: {filename}, Predicted label: {class_labels[predicted_label]}')


1/1 [==============================] - 0s 104ms/step
Image: 1.jpg, Predicted label: Trouser
1/1 [==============================] - 0s 32ms/step
Image: 2.jpg, Predicted label: Bag
1/1 [==============================] - 0s 40ms/step
Image: 3.jpg, Predicted label: Dress
1/1 [==============================] - 0s 60ms/step
Image: 4.jpg, Predicted label: Bag
1/1 [==============================] - 0s 64ms/step
Image: 5.jpg, Predicted label: Bag
1/1 [==============================] - 0s 58ms/step
Image: tyas.jpg, Predicted label: Bag


In [16]:
import tensorflowjs as tfjs
# Path untuk menyimpan model TensorFlow.js
tfjs_model_path = 'tfjs_model_MNIST' #Konversi ke JSON

# Konversi model ke format TensorFlow.js
tfjs.converters.save_keras_model(model, tfjs_model_path)